In [ ]:
"""
Created on Wed Mar 15 10:45 2023

Test Keras Tuner: https://keras.io/guides/keras_tuner/getting_started/

Author: @claraburgard
"""

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner

import xarray as xr

In [ ]:
outputpath = '/bettik/burgardc/DATA/NN_PARAM/interim/NN_MODELS/keras_tuner_tests/'

In [ ]:
lrelu = lambda x: keras.activations.relu(x, alpha=0.1)

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Input(10, name="InputLayer"))
    for i in range(hp.Int("num_layers", 1, 6)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                activation=lrelu,
            )
        )
    model.add(keras.layers.Dense(1, name='Output'))
    model.compile(optimizer="adam", loss='mse', metrics=['mae', 'mse'])
    return model
    

In [ ]:
build_model(keras_tuner.HyperParameters())


In [ ]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_loss",
    max_trials=10,
    executions_per_trial=2,
    overwrite=True,
    directory=outputpath,
    project_name="helloworld",
)


In [ ]:
tuner.search_space_summary()

READ IN THE DATA

In [ ]:
inputpath_data = '/bettik/burgardc/DATA/NN_PARAM/interim/INPUT_DATA/'
inputpath_CVinput = inputpath_data+'EXTRAPOLATED_ISFDRAFT_CHUNKS_CV/'

In [ ]:
norm_method = 'std'

In [ ]:
tblock_dim = range(1,14)
isf_dim = [10,11,12,13,18,22,23,24,25,30,31,33,38,39,40,42,43,44,45,47,48,51,52,53,54,55,58,61,65,66,69,70,71,73,75]

isf_out = 66
tblock_out = 0

data_train_orig_norm = xr.open_dataset(inputpath_CVinput + 'train_data_CV_noisf'+str(isf_out).zfill(3)+'_notblock'+str(tblock_out).zfill(3)+'.nc')
data_val_orig_norm = xr.open_dataset(inputpath_CVinput + 'val_data_CV_noisf'+str(isf_out).zfill(3)+'_notblock'+str(tblock_out).zfill(3)+'.nc') 
data_train_addvar1_norm = xr.open_dataset(inputpath_CVinput + 'train_addvar1_CV_noisf'+str(isf_out).zfill(3)+'_notblock'+str(tblock_out).zfill(3)+'.nc')
data_val_addvar1_norm = xr.open_dataset(inputpath_CVinput + 'val_addvar1_CV_noisf'+str(isf_out).zfill(3)+'_notblock'+str(tblock_out).zfill(3)+'.nc')
    
data_train_norm = data_train_orig_norm[['dGL','dIF','corrected_isfdraft','bathy_metry','slope_bed_lon','slope_bed_lat','slope_ice_lon','slope_ice_lat','theta_in','salinity_in',
                                            'melt_m_ice_per_y']]
data_val_norm = data_val_orig_norm[['dGL','dIF','corrected_isfdraft','bathy_metry','slope_bed_lon','slope_bed_lat','slope_ice_lon','slope_ice_lat','theta_in','salinity_in',
                                        'melt_m_ice_per_y']]

    
## prepare input and target
y_train_norm = data_train_norm['melt_m_ice_per_y'].sel(norm_method=norm_method).load()
x_train_norm = data_train_norm.drop_vars(['melt_m_ice_per_y']).sel(norm_method=norm_method).to_array().load()
    
y_val_norm = data_val_norm['melt_m_ice_per_y'].sel(norm_method=norm_method).load()
x_val_norm = data_val_norm.drop_vars(['melt_m_ice_per_y']).sel(norm_method=norm_method).to_array().load()

In [ ]:
input_size = x_train_norm.values.shape[0]
activ_fct = 'relu' #LeakyReLU
epoch_nb = 100
batch_siz = 512

In [ ]:
tuner.search(x_train_norm.T, y_train_norm, epochs=50, validation_data=(x_val_norm, y_val_norm))